顺序块

Sequential的设计目的是把其他模块串起来。
如果我们要构造自己的简化的MySequential

1. 将块逐个追加到列表中的函数
2. 向前传播函数，用于将输入按照追加块的顺序传递给块组成的“链条”

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, m in enumerate(args):
            self._modules[str(idx)] = m

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [3]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X)

tensor([[-0.0489,  0.0118, -0.1454, -0.1676, -0.1669, -0.0973,  0.0648, -0.1983,
          0.0119,  0.0226],
        [-0.0920, -0.1331, -0.0984,  0.0521, -0.1467, -0.1072,  0.1621, -0.1315,
          0.0720,  0.1611]], grad_fn=<AddmmBackward0>)

只用sequential会不够灵活

我们可以继承nn.Module，然后在
init和forward函数，可以做一些计算
通过这种方式，更加灵活的实现一个模块

In [10]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(), 
                                  nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))
    
Y = torch.rand(2, 20)
testNet = NestMLP()
testNet(Y)

tensor([[-0.1779, -0.2517, -0.0991,  0.0943, -0.0395, -0.0839, -0.1372, -0.0518,
         -0.0049,  0.1129, -0.1744, -0.1157, -0.2409, -0.0598,  0.1941,  0.1265],
        [-0.1827, -0.2062, -0.0663,  0.0967, -0.0052, -0.1178, -0.1052, -0.0580,
          0.0087,  0.1225, -0.1704, -0.1173, -0.2525, -0.0192,  0.2276,  0.1049]],
       grad_fn=<AddmmBackward0>)